Extract Data

Read the CSV file into a Pandas dataframe.

In [1]:
import pandas as pd

# Load dataset
df = pd.read_csv("healthcare_dataset.csv")

# Display first 5 rows
print(df.head())


            Name  Age  Gender Blood Type Medical Condition Date of Admission  \
0  Bobby JacksOn   30    Male         B-            Cancer        2024-01-31   
1   LesLie TErRy   62    Male         A+           Obesity        2019-08-20   
2    DaNnY sMitH   76  Female         A-           Obesity        2022-09-22   
3   andrEw waTtS   28  Female         O+          Diabetes        2020-11-18   
4  adrIENNE bEll   43  Female        AB+            Cancer        2022-09-19   

             Doctor                    Hospital Insurance Provider  \
0     Matthew Smith             Sons and Miller         Blue Cross   
1   Samantha Davies                     Kim Inc           Medicare   
2  Tiffany Mitchell                    Cook PLC              Aetna   
3       Kevin Wells  Hernandez Rogers and Vang,           Medicare   
4    Kathleen Hanna                 White-White              Aetna   

   Billing Amount  Room Number Admission Type Discharge Date   Medication  \
0    18856.281306    

Transform Data

1. Remove Personally Identifiable Information

In [8]:
df = pd.read_csv("healthcare_dataset.csv")


In [9]:
print(df.columns)  # Print all column names


Index(['Name', 'Age', 'Gender', 'Blood Type', 'Medical Condition',
       'Date of Admission', 'Doctor', 'Hospital', 'Insurance Provider',
       'Billing Amount', 'Room Number', 'Admission Type', 'Discharge Date',
       'Medication', 'Test Results'],
      dtype='object')


In [10]:
df.drop(columns=["Name", "Doctor"], inplace=True)

2. Standardize Date Formats

In [11]:
df["Date of Admission"] = pd.to_datetime(df["Date of Admission"]).dt.strftime("%Y-%m-%d")
df["Discharge Date"] = pd.to_datetime(df["Discharge Date"]).dt.strftime("%Y-%m-%d")

3. Handle Missing Values

In [13]:
df["Blood Type"] = df["Blood Type"].fillna("Unknown")
df["Medical Condition"] = df["Medical Condition"].fillna("Unknown")
df["Billing Amount"] = df["Billing Amount"].fillna(df["Billing Amount"].mean())
df["Test Results"] = df["Test Results"].fillna("Inconclusive")

4. Standardize Categorical Values

In [14]:
df["Gender"] = df["Gender"].replace({"Male": "M", "Female": "F"})
df["Admission Type"] = df["Admission Type"].str.title()  # Ensure proper casing

5. Convert Currency Format

In [15]:
df["Billing Amount"] = df["Billing Amount"].astype(float)

Connect Python to MySQL

In [1]:
import mysql.connector
import pandas as pd

try:
    # Connect to MySQL without a password
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  # Empty string for no password
        database="healthcare_db"
    )

    if conn.is_connected():
        print("Connected to MySQL")

        # Create a cursor
        with conn.cursor() as cursor:
            # Check connection and fetch MySQL version
            cursor.execute("SELECT VERSION()")
            mysql_version = cursor.fetchone()[0]
            print("MySQL Version:", mysql_version)

except mysql.connector.Error as err:
    print("Error:", err)

finally:
    if 'conn' in locals() and conn.is_connected():
        conn.close()
        print("Connection closed.")



Connected to MySQL
MySQL Version: 5.6.31
Connection closed.


Load Data from CSV to MySQL

In [3]:
import mysql.connector
import pandas as pd

def load_to_mysql(df, conn, table_name):
    try:
        with conn.cursor() as cursor:
            # SQL query for inserting records
            insert_query = f"""
            INSERT INTO {table_name} 
            (age, gender, blood_type, medical_condition, date_of_admission, hospital, insurance_provider, billing_amount,
            room_number, admission_type, discharge_date, medication, test_results)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """

            # Handle NaN values to avoid insertion issues
            df = df.fillna("NULL")  # Replace with a default value (adjust if needed)

            # Convert DataFrame rows to list of tuples
            data = [tuple(row) for row in df.itertuples(index=False, name=None)]

            # Insert records into MySQL
            cursor.executemany(insert_query, data)
            conn.commit()
            print(f"✅ Successfully loaded {len(df)} records into {table_name}!")

    except mysql.connector.Error as err:
        conn.rollback()  # Rollback in case of an error
        print(f"Error loading data: {err}")

    finally:
        if conn.is_connected():
            conn.close()
            print("MySQL connection closed.")

# Establish MySQL connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",  # Empty if no password is set
    database="healthcare_db"
)

# Read CSV and Load into MySQL
df = pd.read_csv("healthcare_dataset.csv")
load_to_mysql(df, conn, "healthcare_records")


Error loading data: Not all parameters were used in the SQL statement
MySQL connection closed.
